## Preliminar analysis on closedloop delays

In [1]:
import mne
from mne.preprocessing import ICA
from closedloop.data.utils.utils import read_elc
import numpy as np
import os
import pyprep
import matplotlib
matplotlib.use('Qt5Agg')
import matplotlib.pyplot as plt


In [2]:
# Load raw data
vhdr_fname = '/media/jerry/ruggero/closedloop_rec/CL001/N1/eeg/CL001_N1_2025-02-27_00-53-52.vhdr'
# eog = ['EOG']
raw  = mne.io.read_raw_brainvision(vhdr_fname, 
                                   misc=['BIP1', 'BIP2', 'BIP3'], 
                                   scale=1.,
                                   preload=True)

Extracting parameters from /media/jerry/ruggero/closedloop_rec/CL001/N1/eeg/CL001_N1_2025-02-27_00-53-52.vhdr...
Setting channel info structure...
Reading 0 ... 10508618  =      0.000 ... 21017.236 secs...


/tmp/ipykernel_13102/3814735828.py:4: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw  = mne.io.read_raw_brainvision(vhdr_fname,


In [3]:
# Load electrode locations
elc_fname = '/media/jerry/ruggero/closedloop_rec/CL001/N1/eeg/CL001_CA-212_26-02-2025_23-24.elc'
montage = read_elc(elc_fname)

In [ ]:
# Preprocessing
raw.add_reference_channels('5Z')
raw.rename_channels(mapping={'BIP1': 'EMG',
                             'BIP2': 'ECG',
                             'BIP3': 'RES'})
raw.set_channel_types(mapping={'EMG': 'emg',
                               'ECG': 'ecg',
                               'RES': 'resp'})
raw.set_montage(montage)
# Referencing and renaming the vertical ocular channel
raw = mne.set_bipolar_reference(raw, 'EOG', '1L', 
                                ch_name='VEOG',
                                drop_refs=False, 
                                copy='False')
# Deleting the old unreferenced VEOGR channel
raw.drop_channels(ch_names=['EOG'])
# Creating the horizontal ocular channel 
raw = mne.set_bipolar_reference(raw, '1RD', '1LD', 
                                ch_name='HEOG', 
                                drop_refs=False, 
                                copy='False')
# Assing type to ocular channels
raw.set_channel_types(mapping={'VEOG': 'eog',
                               'HEOG': 'eog'})

raw = raw.notch_filter(freqs=np.arange(50, 201, 50), n_jobs='cuda')

raw = raw.filter(l_freq=.2, h_freq=None, picks='eeg', n_jobs='cuda')
raw = raw.filter(l_freq=.5, h_freq=40., picks='ecg', n_jobs='cuda')
raw = raw.filter(l_freq=10., h_freq=None, picks='emg', n_jobs='cuda')
raw = raw.filter(l_freq=.1, h_freq=30., picks='eog', n_jobs='cuda')

nc = pyprep.NoisyChannels(raw=raw.copy().pick_types(eeg=True).
                            resample(100.), 
                            do_detrend=True, random_state=23, 
                            matlab_strict=False)
nc.find_all_bads()
nc.find_bad_by_nan_flat()
bads = nc.get_bads(as_dict=True)

raw.info['bads'] = bads['bad_all']
print('Channels marked as bad:', raw.info['bads'])


# raw.info['bads'] = ['2LB', '5Z']
# raw.interpolate_bads(reset_bads=True)
# raw.filter(0.3, 30)
# raw.set_eeg_reference(['3LD', '3RD'], projection=False)

/tmp/ipykernel_13102/2418047447.py:6: RuntimeWarning: The unit for channel(s) ECG, EMG, RES has changed from NA to V.
  raw.set_channel_types(mapping={'EMG': 'emg',


EEG channel type selected for re-referencing
Creating RawArray with float64 data, n_channels=1, n_times=10508619
    Range : 0 ... 10508618 =      0.000 ... 21017.236 secs
Ready.


2025-02-28 13:27:04,690 - numexpr.utils - INFO - NumExpr defaulting to 16 threads.


Added the following bipolar channels:
VEOG
EEG channel type selected for re-referencing
Creating RawArray with float64 data, n_channels=1, n_times=10508619
    Range : 0 ... 10508618 =      0.000 ... 21017.236 secs
Ready.
Added the following bipolar channels:
HEOG
Filtering raw data in 1 contiguous segment
Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower transition bandwidth: 0.50 Hz
- Upper transition bandwidth: 0.50 Hz
- Filter length: 3301 samples (6.602 s)

Now using CUDA device 0
Enabling CUDA with 5.42 GiB available memory
Using CUDA for FFT FIR filtering
Filtering raw data in 1 contiguous segment
Setting up high-pass filter at 0.2 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal highpass filter:
- Windowed time-domain desig

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.6s


Executing RANSAC
This may take a while, so be patient...


  0%|          |  : 0/4203 [00:00<?,       ?it/s]


RANSAC done!
Channels marked as bad: ['2LB', '3RD', '7Z', '6L', '5R', '3LA', '5Z']


: 

In [ ]:
filt_raw = raw.copy().filter(1., 45., n_jobs='cuda')

components = (len(raw.get_channel_types(picks='eeg')) - len(raw.info['bads']))

ica = ICA(n_components=components, noise_cov=None, random_state=23, 
          method='fastica', max_iter=1500, allow_ref_meg=False)

ica.fit(inst=filt_raw, decim=5, reject=None)

bad_ecg, ecg_scores = ica.find_bads_ecg(filt_raw, ch_name='ECG')
bad_emg, emg_scores = ica.find_bads_muscle(filt_raw, sphere='auto',
                                           threshold=0.1)

bads_ica = list(np.unique(bad_ecg + bad_emg))

ica.exclude = bads_ica

del filt_raw

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 45.00 Hz
- Upper transition bandwidth: 11.25 Hz (-6 dB cutoff frequency: 50.62 Hz)
- Filter length: 1651 samples (3.302 s)

Using CUDA for FFT FIR filtering
Fitting ICA to data using 57 channels (please be patient, this may take a while)
Selecting by number: 57 components


/tmp/ipykernel_13102/708974406.py:6: RuntimeWarning: Using n_components=57 (resulting in n_components_=57) may lead to an unstable mixing matrix estimation because the ratio between the largest (56) and smallest (1.5e-05) variances is too large (> 1e6); consider setting n_components=0.999999 or an integer <= 47
  ica.fit(inst=filt_raw, decim=5, reject=None)


Fitting ICA took 280.8s.
Using threshold: 0.23 for CTPS ECG detection
Using channel ECG to identify heart beats.
Setting up band-pass filter from 8 - 16 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 8.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 7.75 Hz)
- Upper passband edge: 16.00 Hz
- Upper transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 16.25 Hz)
- Filter length: 5000 samples (10.000 s)

Number of ECG events detected : 22621 (average pulse 64.57841891498778 / min.)
Not setting metadata
22621 matching events found
No baseline correction applied
Using data from preloaded Raw for 22621 events and 501 original time points ...
1 bad epochs dropped


In [ ]:
prep_raw = raw.copy()
prep_raw = prep_raw.filter(.2, 45., n_jobs='cuda')
prep_raw = ica.apply(prep_raw, exclude=bads_ica)
prep_raw = prep_raw.interpolate_bads(reset_bads=True, 
                                     mode='accurate', 
                                     method='spline')
prep_raw.set_eeg_reference(['3LD', '3RD'])

nc = pyprep.NoisyChannels(raw=prep_raw.copy().pick_types(eeg=True).
                                resample(100.), 
                                do_detrend=True, random_state=23,
                                matlab_strict=False)
nc.find_all_bads()
bads = nc.get_bads(as_dict=True)
prep_raw.info['bads'] = bads['bad_all']
print('Channels marked as bad:', prep_raw.info['bads'])
prep_raw = prep_raw.interpolate_bads(reset_bads=True, 
                                        mode='accurate', 
                                        method='spline')

In [ ]:
ev_id = {'Stimulus/s1': 1,
         'Stimulus/s22': 22,
         'Stimulus/s24': 24,
         'Stimulus/s32': 32,
         'Stimulus/s34': 34,
         'Stimulus/s38': 38}
events, ev_dict = mne.events_from_annotations(prep_raw, ev_id)

Used Annotations descriptions: ['Stimulus/s1', 'Stimulus/s22', 'Stimulus/s24', 'Stimulus/s32', 'Stimulus/s34', 'Stimulus/s38']


In [ ]:
deletion_list = []
for i, e in enumerate(events):
    if i == 0:
        if e[2] == 1:
            deletion_list.append(i)
    else:
        if e[2] == 1 and events[i-1][2] not in [22, 24, 32, 34]:
            deletion_list.append(i)
events = np.delete(events, deletion_list, axis=0)

In [ ]:
new_events = []
for i, e in enumerate(events):
    if e[2] == 1:
        if i <= 1:
            if events[i+1][2] == 38:
                new_events.append([e[0], 0, events[i-1][2]])
            else:
                new_events.append([e[0], 0, events[i-1][2]+1])
        else:
            if events[i+1][2] == 38:
                if events[i-2][2] == 38:
                    new_events.append([e[0], 0, events[i-1][2]])
            else:
                if events[i-2][2] == 38:
                    new_events.append([e[0], 0, events[i-1][2]+1])

In [ ]:
new_ev_id = {'cin_lh_neg': 22,
             'cin_rh_neg': 24,
             'occ_lh_neg': 32,
             'occ_rh_neg': 34,
             'cin_lh_pos': 23,
             'cin_rh_pos': 25,
             'occ_lh_pos': 33,
             'occ_rh_pos': 35}

epochs = mne.Epochs(prep_raw, events=new_events, event_id=new_ev_id, tmin=-3.5, 
                    tmax=1.5, baseline=(-3.5, -1.5), preload=True)
epochs.filter(0.5, 4)

Not setting metadata
50 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 50 events and 2501 original time points ...
0 bad epochs dropped
Setting up band-pass filter from 0.5 - 4 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 4.00 Hz
- Upper transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 5.00 Hz)
- Filter length: 3301 samples (6.602 s)



/tmp/ipykernel_8377/4055589267.py:12: RuntimeWarning: filter_length (3301) is longer than the signal (2501), distortion is likely. Reduce filter length or filter a longer signal.
  epochs.filter(0.5, 4)
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 161 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 287 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done 449 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done 647 tasks      | elapsed:    0.2s
[Parallel(n_jobs=1)]: Done 881 tasks      | elapsed:    0.3s
[Parallel(n_jobs=1)]: Done 1151 tasks      | elapsed:    0.3s
[Parallel(n_jobs=1)]: Done 1457 tasks      | elapsed:    0.4s
[Parallel(n_jobs=1)]: Done 1799 tasks      | elapsed:    0.5s
[Parallel(n_jobs=1)]: Done 2177 tasks      | elapsed:    0.6s
[Parallel(n_jobs=1)]: Done 2591 tasks      | elapsed:    0.8s
[Parallel(n_jobs=1)]: Done 3041 tasks      | elapsed:    0.9

<Epochs | 50 events (all good), -3.5 – 1.5 s (baseline -3.5 – -1.5 s), ~66.0 MiB, data loaded,
 'cin_lh_neg': 13
 'cin_rh_neg': 6
 'occ_lh_neg': 6
 'occ_rh_neg': 3
 'cin_lh_pos': 7
 'cin_rh_pos': 5
 'occ_lh_pos': 8
 'occ_rh_pos': 2>

In [ ]:
epochs.plot_image()

Not setting metadata
50 matching events found
No baseline correction applied
0 projection items activated
combining channels using GFP (eeg channels)


[<Figure size 640x480 with 3 Axes>]

In [ ]:
epochs['cin_rh_neg'].average().plot()

QCoreApplication::exec: The event loop is already running


<Figure size 640x300 with 2 Axes>